In [24]:

import nltk
from nltk.tokenize import WhitespaceTokenizer


tokenizer = WhitespaceTokenizer()

nltk.download()

from nltk.corpus import stopwords
stopword_list = stopwords.words('english')
print stopword_list 

import pandas as pd
reviews_df = pd.read_csv(r"C:\Users\Deon\test_chunk1.csv", encoding="utf-8") #replace with smaller chunks
#reviews_df 

positive_terms = []
f = open('C:\Users\Deon\Downloads\positive_terms.txt', "r")
positive_terms = f.read().splitlines()
f.close()

negative_terms = []
f = open(r'C:\Users\Deon\Downloads\negtive_terms.txt', "r")
negative_terms = f.read().splitlines()
f.close()

print positive_terms
print ""
print negative_terms

porter = nltk.PorterStemmer()



showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both'

In [25]:
import string

def remove_punctuation(text):
    #this removes punctuations that are defined in the string library
    punctuations = string.punctuation 

    # But don't strip out apostrophes
    excluded_punctuations = ["'"]
    for p in punctuations:
        if p not in excluded_punctuations:
            # replace each punctuation symbol by a space
            text = text.replace(p, " ") 

    return text

porter = nltk.PorterStemmer()

In [26]:


def normalize_review_text(text):
    #text = text.lower()
    text = remove_punctuation(text)
    text = " ".join(text.split())
    text_tokens = tokenizer.tokenize(text)
    text_tokens = [porter.stem(w) for w in text_tokens if w not in stopword_list]
    return text_tokens

# Apply the function above to the text column
reviews_df["text"] = reviews_df["text"].apply(normalize_review_text)
reviews_df

def calculate_positivity(text):
    num_tokens = len(text)
    num_positive_tokens = 0
    for t in text:
        if t in positive_terms:
            num_positive_tokens = num_positive_tokens + 1
    # The positivity score is the fraction of tokens that were found in the positive dictionary
    return float(num_positive_tokens) / float(num_tokens)

reviews_df["positivity_score"] = reviews_df["text"].apply(calculate_positivity)



In [27]:
def calculate_negativity(text):
    num_tokens = len(text)
    num_negative_tokens = 0
    for t in text:
        if t in negative_terms:
            num_negative_tokens = num_negative_tokens + 1
    # The positivity score is the fraction of tokens that were found in the positive dictionary
    return float(num_negative_tokens) / float(num_tokens)

reviews_df["negativity_score"] = reviews_df["text"].apply(calculate_negativity)
reviews_df

import scipy.stats as sp

pos_score_stars_corr = sp.pearsonr(reviews_df["stars"].values, reviews_df["positivity_score"].values)
pos_score_stars_corr

print ""

neg_score_stars_corr = sp.pearsonr(reviews_df["stars"].values, reviews_df["negativity_score"].values)
neg_score_stars_corr


(0.093613417686275804, 0.69464512288309499)

In [28]:
print reviews_df["positivity_score"]

0     0.093023
1     0.062500
2     0.125000
3     0.169811
4     0.069444
5     0.102564
6     0.071429
7     0.090909
8     0.137500
9     0.081633
10    0.046512
11    0.035714
12    0.041667
13    0.060000
14    0.046875
15    0.060606
16    0.052632
17    0.029412
18    0.000000
19    0.033613
Name: positivity_score, dtype: float64


In [29]:
print reviews_df["negativity_score"]

0     0.000000
1     0.062500
2     0.000000
3     0.000000
4     0.000000
5     0.000000
6     0.023810
7     0.000000
8     0.000000
9     0.040816
10    0.007752
11    0.000000
12    0.010417
13    0.000000
14    0.031250
15    0.000000
16    0.000000
17    0.014706
18    0.020833
19    0.000000
Name: negativity_score, dtype: float64
